# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298635879753                   -0.85    5.9    239ms
  2   -8.300221202907       -2.80       -1.25    2.0    100ms
  3   -8.300437489216       -3.66       -1.85    1.9    100ms
  4   -8.300459388301       -4.66       -2.62    1.0   88.6ms
  5   -8.300464233595       -5.31       -2.99    2.5    125ms
  6   -8.300464585732       -6.45       -3.21    1.5    100ms
  7   -8.300464620833       -7.45       -3.47    1.1   87.4ms
  8   -8.300464632499       -7.93       -3.66    1.6   96.9ms
  9   -8.300464637741       -8.28       -3.79    1.0   92.2ms
 10   -8.300464641814       -8.39       -4.02    1.0   84.8ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67559237249                   -0.70    6.1    495ms
  2   -16.67878774751       -2.50       -1.14    1.0    209ms
  3   -16.67921713692       -3.37       -1.87    3.0    262ms
  4   -16.67927180789       -4.26       -2.74    3.4    269ms
  5   -16.67928547177       -4.86       -3.20    3.5    323ms
  6   -16.67928614692       -6.17       -3.49    3.2    283ms
  7   -16.67928620672       -7.22       -3.89    1.2    199ms
  8   -16.67928621938       -7.90       -4.45    2.5    258ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32798303824                   -0.56    7.1    1.44s
  2   -33.33469394614       -2.17       -1.00    1.2    699ms
  3   -33.33600461690       -2.88       -1.73    6.4    1.09s
  4   -33.33615300035       -3.83       -2.65    2.5    738ms
  5   -33.33687631450       -3.14       -2.43    9.5    1.34s
  6   -33.33690531016       -4.54       -2.56    1.1    673ms
  7   -33.33694323614       -4.42       -3.70    1.0    650ms
  8   -33.33694373997       -6.30       -3.90    5.5    1.22s
  9   -33.33694376984       -7.52       -4.15    1.9    978ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298808530402                   -0.85    5.4    264ms
  2   -8.300299561836       -2.83       -1.59    1.1   85.3ms
  3   -8.300442106491       -3.85       -2.71    2.1    100ms
  4   -8.300446694866       -5.34       -2.62    3.5    143ms
  5   -8.300464292998       -4.75       -3.34    1.0   77.8ms
  6   -8.300464595073       -6.52       -3.79    1.5   94.2ms
  7   -8.300464641323       -7.33       -4.45    1.8    100ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32824424178                   -0.56    6.6    1.41s
  2   -33.26659191305   +   -1.21       -1.24    1.2    596ms
  3   +18.26064631609   +    1.71       -0.21   10.5    1.99s
  4   -33.29194476632        1.71       -1.59    6.1    1.58s
  5   -33.23859692795   +   -1.27       -1.34    3.1    925ms
  6   -33.01930309210   +   -0.66       -1.31    3.4    921ms
  7   -33.15939335195       -0.85       -1.43    4.1    987ms
  8   -33.33362189285       -0.76       -2.24    2.8    772ms
  9   -33.33641249508       -2.55       -2.63    4.2    1.03s
 10   -33.33683529150       -3.37       -2.79    2.9    816ms
 11   -33.33687939395       -4.36       -2.95    2.2    761ms
 12   -33.33692854441       -4.31      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.